In [2]:
import tensorflow as tf
from tensorflow.examples.tutorials.mnist import input_data

In [3]:
mnist = input_data.read_data_sets("MNIST_data/", one_hot=True)

Extracting MNIST_data/train-images-idx3-ubyte.gz
Extracting MNIST_data/train-labels-idx1-ubyte.gz
Extracting MNIST_data/t10k-images-idx3-ubyte.gz
Extracting MNIST_data/t10k-labels-idx1-ubyte.gz


# Helper Functions

In [31]:
def init_weight(shape):
    # shape = [H, W, C, filters]
    init_random_dist = tf.truncated_normal(shape, stddev=0.1)
    return tf.Variable(init_random_dist)

def init_bias(shape):
    init_bias = tf.constant(shape=shape, value=0.1)
    
    return tf.Variable(init_bias)

def conv2d(x, W):
    # x.shape = [batch, H, W, Depth]
    # W.shape = [H, W, Depth, NumofFilter]
    return tf.nn.conv2d(x, W, strides=[1,1,1,1], padding="SAME")

def max_pool_2by2(x):
    # x.shape = [batch, H, W, NumofFilter]
    return tf.nn.max_pool(x, ksize=[1, 2, 2, 1], strides=[1, 2, 2, 1], 
                          padding="SAME")

def convolutional_layer(input_x, shape):
    W = init_weight(shape)
    b = init_weight([shape[3]])
    return tf.nn.relu(conv2d(input_x, W) + b)

def fully_connected_layer(input_layer, size):
    input_size = int(input_layer.get_shape()[1])
    W = init_weight([input_size, size])
    b = init_bias([size])
    
    return tf.matmul(input_layer, W) + b

# PlaceHolder

In [41]:
x = tf.placeholder(dtype=tf.float32, shape=[None, 784])
y_true = tf.placeholder(dtype=tf.float32, shape=[None, 10])

# Layers

In [42]:
x_image = tf.reshape(x, shape=[-1, 28, 28, 1])

convo_1 = convolutional_layer(x_image, shape=[6, 6, 1, 32])
convo_1_pooling = max_pool_2by2(convo_1)

convo_2 = convolutional_layer(convo_1_pooling, shape=[6, 6, 32, 64])
convo_2_pooling = max_pool_2by2(convo_2)

convo_2_flat = tf.reshape(convo_2_pooling, [-1, 7 * 7 * 64])
full_layer_one = tf.nn.relu(fully_connected_layer(convo_2_flat, 1024))

hold_prob = tf.placeholder(tf.float32)
dropout = tf.nn.dropout(x=full_layer_one, keep_prob=hold_prob)
y_pred = fully_connected_layer(dropout, 10)

# Train Functions

In [46]:
cross_entropy = tf.reduce_mean(tf.nn.softmax_cross_entropy_with_logits(labels=y_true, logits=y_pred))
optimizer = tf.train.AdamOptimizer(learning_rate=0.0001)
train = optimizer.minimize(cross_entropy)

init = tf.global_variables_initializer()

# Session 

In [ ]:
steps = 5000
with tf.Session() as sess:
    sess.run(init)
    
    for i in range(steps):
        batch_x, batch_y = mnist.train.next_batch(50)
        sess.run(train, feed_dict={x:batch_x, y_true:batch_y, hold_prob:0.5})
        
        if i % 100 == 0:
            print('Currently on step {}'.format(i))
            print('Accuracy is:')
            matches = tf.equal(tf.argmax(y_pred, 1), tf.argmax(y_true, 1))
            acc = tf.reduce_mean(tf.cast(matches, tf.float32))
            
            print (sess.run(acc, feed_dict={x:mnist.test.images, y_true:mnist.test.labels, hold_prob:1.0}))

Currently on step 0
Accuracy is:
0.05
Currently on step 100
Accuracy is:
